# Determine mortality crises

This notebook takes the results generated in "AnalyzeExcessMortality.ipynb" and identifies extended periods with excess mortality and groups them together as "mortality crises". 

The results are collected in a single dataframe, which is saved as a csv file.

In [18]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib widget

# Load style
plt.style.use('PlotStyle.mplstyle')
import matplotlib.colors as colors
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)

from datetime import datetime
from tqdm import tqdm

import os

# Load functions
import sys
sys.path.append("../../ExcessMortality")
import ExcessMortalityFunctions as emf
import AdditionalFunctions as ps


# saveFigures = True
# # saveFigures = False
# print('saveFigures is set to: '+str(saveFigures))
print('Done loading packages')

Done loading packages


In [19]:
# Set paths
pathData = '../Data/'
# pathResults = '../../AnalysisResults' # NOTE: Outside repo! This is done to save space within repo. In the final part of the analysis, everything is put together in a single file
pathResults = '../Data/AnalysisResults' 

In [20]:
dfRel = pd.read_csv('../SupplementaryTable_RelationalTable_ParishCounty.csv')
dfRel['StartDate'] = pd.to_datetime(dfRel.StartDate)
dfRel['EndDate'] = pd.to_datetime(dfRel.EndDate)

In [21]:
# Table with population data
dfPop = pd.read_excel(pathData + 'AmtPopulation.xlsx')
dfPop['DigDagID'] = dfPop['DigDagID'].fillna(0).astype(int) # Since excel sometimes saves ints as floats

In [22]:
# Only consider counties that exists after 1810 
# (A large restructuring was done between 1800 and 1810)
# print(dfRel.AmtID.unique())
print(len(dfRel.AmtID.unique()))
allAmtIDs = dfRel[dfRel.EndDate > np.datetime64('1810-01-01')].AmtID.unique()
# print(allAmtIDs)
print(len(allAmtIDs))

46
27


# Parameters used in main analysis

In [23]:
# Flags and analysis parameters used in main analysis 
numYears = 6 # Number of years on both sides of date to use for baseline calculations 
numYears = 12 # Number of years on both sides of date to use for baseline calculations 
numYearsTot = (numYears*2) # The "name" of the baseline (i.e. +/- 5 years is a 10-year baseline, +/- 12 is a 24 year baseline)
thresholdExcess = 3 # Threshold (in terms of Z-scores) for identifying a day as having increased excess

# Determine directory in which results was saved
pathResultsUpper = pathResults + f'_Years{numYears}_Threshold{thresholdExcess}/'


In [24]:
# Define the agegroups analyzed
ageGroups = [
    ['Total'],
    ['Stillborn','0'],
    ['1-4','5-9', '10-14'],
    ['15-19', '20-24', '25-29', '30-34', '35-39'],
    ['40-44', '45-49', '50-54', '55-59'],
    ['60-64', '65-69', '70-74', '75-79', '80+']
]

# And the names used for directories and filenames
ageGroupNames = [
    'Total',
    'Infants_stillborn',
    '1-14',
    '15-39',
    '40-59',
    '60+'
]

# Parameters for identification of mortality crises

In [25]:
# Parameters to use here
thresholdLower = 2 # Lower threshold used for determining the start and end of periods (in terms of Z-scores)
maxDaysBelowThreshold = 4 # Number of days below thresholdLower before a period of excess is "stopped"
# maxDaysBelowThreshold = 7 # Number of days below thresholdLower before a period of excess is "stopped"
minimumLengthOfEpidemic = 4 # Minimal number of days above thresholdExcess which is counted as a period of excess 

excessCountThreshold = 50 # Only save mortality crises with more than this number of excess deaths
excessCountThreshold = 20 # Only save mortality crises with more than this number of excess deaths

# Determine filename to use for final results
finalResultsFilename = 'AllCrises'+f'_Years{numYears}_Threshold{thresholdExcess}_LowerThreshold{thresholdLower}_MaxDaysBelow{maxDaysBelowThreshold}_minLength{minimumLengthOfEpidemic}_minCount{excessCountThreshold}'
finalResultsFilename

'AllCrises_Years12_Threshold3_LowerThreshold2_MaxDaysBelow4_minLength4_minCount20'

# Helper functions for saving season and quarter

In [26]:
def getSeason(d):
    curMonth = pd.to_datetime(d).month
    
    seasonDict = {
        1:'Winter',
        2:'Winter',
        3:'Spring',
        4:'Spring',
        5:'Spring',
        6:'Summer',
        7:'Summer',
        8:'Summer',
        9:'Fall',
        10:'Fall',
        11:'Fall',
        12:'Winter',
    }
    return seasonDict[curMonth]
    
def getQuarter(d):
    curMonth = pd.to_datetime(d).month
    
    QuarterDict = {
        1:'Q1',
        2:'Q1',
        3:'Q1',
        4:'Q2',
        5:'Q2',
        6:'Q2',
        7:'Q3',
        8:'Q3',
        9:'Q3',
        10:'Q4',
        11:'Q4',
        12:'Q4',
    }
    return QuarterDict[curMonth]

# Functions for interpolation of population counts

In [27]:
def getPopRow(amtID):
    curRow = dfPop[dfPop.DigDagID == amtID]
    if len(curRow) == 0:
        print(amtID)
        print('----- Amt not found in population table -----')

    elif len(curRow) > 1:
        # print(f'----- Amt ID {amtID} gives too many results -----')        
        return curRow
    else:
        return curRow 

def AmtIDToDailyPopulation(amtID,dateToFocusOn=np.datetime64('1870-01-01')):
    
    # Get current row from population table
    curPopRow = getPopRow(amtID)
    # Remove name and ID
    curPopRow = curPopRow.iloc[:,2:]

    # Special case for Århus Amt (for deciding whether to include Skanderborg Amt counts or not)
    if (amtID == 118846):
        if dateToFocusOn >= np.datetime64('1867-01-01'):
            curPopRow = curPopRow.iloc[1:]
        else:
            curPopRow = curPopRow.iloc[:1]
            
    # Transpose and rename
    curdfPop = curPopRow.T.reset_index()
    curdfPop = curdfPop.rename(columns={curdfPop.columns[0]:'Year',curdfPop.columns[1]:'Population'})

    # Get years-values as dates        
    curdfPop['Date'] = [np.datetime64(str(x)+'-01-01') for x in curdfPop.Year]

    # Make a dataframe with all dates
    dfAllDates = pd.DataFrame(
        {'Date':np.arange(curdfPop.Date.iloc[0],curdfPop.Date.iloc[-1],np.timedelta64(1,'D'))}
    ) 

    # Merge with empty
    curdfPop = pd.merge(curdfPop,dfAllDates,on='Date',how='outer').sort_values('Date').drop(columns='Year')

    ## New addition during review: Exponential interpolation rather than linear
    # Log population counts before interpolation
    curdfPop['Population'] = np.log(curdfPop['Population'])

    # Carry out interpolation
    curdfPopInterpolate = curdfPop.set_index('Date').interpolate()

    ## New addition during review: Exponential interpolation rather than linear
    # Take exponential of interpolated number 
    curdfPopInterpolate['Population'] = np.exp(curdfPopInterpolate['Population'])
    curdfPop['Population'] = np.exp(curdfPop['Population'])
    
    return curdfPop,curdfPopInterpolate


# Run through all saved files, identify crises and collect them in a dataframe

In [28]:
# Define dataframe to collect results in 
# dfCrisesCollect = pd.DataFrame(
#     columns = [
#         'Amt',
#         'Start',
#         'End',
#         'NumberOfDays',
#         'DayWithMostBurials',
#         'Excess',
#         'ExcessPct',
#         'GenderRatio',
#         'TimeOfYear',
#         'Season'
#     ]
# ) 

dfCrisesCollectAge = pd.DataFrame(
    columns = [
        'Amt',
        'Start',
        'End',
        'NumberOfDays',
        'DayWithMostBurials',
        'Excess',
        'ExcessPct',
        'GenderRatio',
        'TimeOfYear',
        'Season',
        'PopulationEstimate',
        'Exc_Infants_stillborn',
        'Exc_1-14',
        'Exc_15-39',
        'Exc_40-59',
        'Exc_60+',
        'Pct_Infants_stillborn',
        'Pct_1-14',
        'Pct_15-39',
        'Pct_40-59',
        'Pct_60+',
        'DataSum_Infants_stillborn',
        'DataSum_1-14',
        'DataSum_15-39',
        'DataSum_40-59',
        'DataSum_60+',
        'Baseline_Infants_stillborn',
        'Baseline_1-14',
        'Baseline_15-39',
        'Baseline_40-59',
        'Baseline_60+',
    ]
) 


In [29]:
# Prepare progressbar and go through all counties
pbar = tqdm(allAmtIDs)
for curAmtID in pbar:

    # Get county name, data and periods
    curAmtName = ps.getAmtName(curAmtID)
    alldfs,allStarts,allEnds = ps.getAmtCollections(curAmtID)


    # Go through each possible period
    for i in range(len(allStarts)):

        # Get the dataframe, start date and end date
        curdf = alldfs[i].copy()
        curStart = allStarts[i]
        curEnd = allEnds[i]

        # Make sure that the "Date" columns in the data-dataframe is a datetime64 object
        curdf['Date'] = pd.to_datetime(curdf['Date'])

        # Update progressbar
        pbar.set_postfix(
            {
                'Amt':curAmtName,
                'Period':i,
                'Total periods':len(allStarts),
                'Start':curStart,
                'End':curEnd,
            }
        )

        # Determine filename results-file
        curFileName =  str(int(curAmtID)) + '_'+curAmtName + '_'+pd.to_datetime(curStart).strftime('%Y-%m-%d') +'_'+pd.to_datetime(curEnd).strftime('%Y-%m-%d')
        # curFileName = curFileName + '_Total.csv' # Use results from analysis of all ages
        curFileName = curFileName + '.csv' 

        # Load analysis file
        dfPeriod = pd.read_csv(pathResultsUpper+curFileName)
        # Make sure date is a datetime64 object
        dfPeriod['Date'] = pd.to_datetime(dfPeriod.Date)

        # Restrict to valid period 
        dfPeriod = dfPeriod[(dfPeriod.Date >= curStart) & (dfPeriod.Date < curEnd)].reset_index(drop=True)

        # Get results and re-calculate excess
        curTime = dfPeriod.Date 
        curExcess = dfPeriod.Total_Data7DayMean - dfPeriod.Total_Baseline 
        curZscore = dfPeriod.Total_Zscore 
        
        # Use function from ExcessMortalityFunctions to determine mortality crisis period
        dateGroups,allExcess  = emf.determineMortalityCrisis(curTime,curExcess,curZscore,upperThreshold=thresholdExcess,lowerThreshold=thresholdLower,maxDaysBelowThreshold=maxDaysBelowThreshold,minimumLengthOfEpidemic=minimumLengthOfEpidemic,returnExcessCount=True)
        
        # Go through each mortality crisis
        for excID in range(len(dateGroups)):
            
            # Get current start, end and total excess
            curGroup = dateGroups[excID]
            curExc = allExcess[excID]

            # If the period is significant enough
            if (curExc >= excessCountThreshold):
                
                # Get start and end
                curCrisisStart = curGroup[0]
                curCrisisEnd = curGroup[1]

                # Determine duraction (in days)
                curDuration = int((curCrisisEnd - curCrisisStart)/np.timedelta64(1,'D'))

                # Calculate gender ratio of all deaths in period from data-collection dataframe
                dfDataCrisis = curdf[(curdf.Date >= curCrisisStart) & (curdf.Date <= curCrisisEnd)]
                GenderRatioInPeriod = dfDataCrisis.Male.sum()/(dfDataCrisis.Male.sum()+dfDataCrisis.Female.sum())

                # Determine the date (during the crisis) where raw data is highest.
                curDeadliestDay = dfDataCrisis.iloc[dfDataCrisis.Total.argmax()]['Date']
                
                # Make a dataframe consisting of analysis-results only during period
                dfCrisis = dfPeriod[(dfPeriod.Date >= curCrisisStart) & (dfPeriod.Date <= curCrisisEnd)]

                # Total deaths in period, data
                curTotData = dfCrisis.Total_Data7DayMean.sum()
                # Total deaths in period, baseline
                curTotBase = dfCrisis.Total_Baseline.sum()
                # Excess deaths in entire period, in percent
                curExcPct = (curTotData - curTotBase)/curTotBase
                # curExcPct = int(np.round(100 * curExcPct))
                curExcPct = np.round(100 * curExcPct)

                # Get population estimate (rounded to an integer)
                _,populationInterpolated = AmtIDToDailyPopulation(curAmtID,curDeadliestDay)
                curPopulation = np.round(populationInterpolated.loc[curDeadliestDay]['Population'])

                # Collect results as a row to add to dataframe
                curRowToAdd = pd.Series({
                    'Amt': curAmtName,
                    'Start': curCrisisStart,
                    'End': curCrisisEnd,
                    'NumberOfDays': curDuration,
                    'DayWithMostBurials': curDeadliestDay,
                    'Excess': int(np.round(curExc)),
                    'ExcessPct': curExcPct,
                    'GenderRatio': GenderRatioInPeriod,
                    'TimeOfYear': getQuarter(curDeadliestDay),
                    'Season': getSeason(curDeadliestDay),
                    'PopulationEstimate': curPopulation,
                })

                # # Add row to primary dataframe
                # dfCrisesCollect.loc[len(dfCrisesCollect)] = curRowToAdd
            
                #### Get results of age-specific analysis
                # Go through each agegroup
                for ageIndex in range(len(ageGroups)):
                    
                    # Get the agegroup and the name of the group
                    curAgeGroup = ageGroups[ageIndex]
                    curAgeName = ageGroupNames[ageIndex]

                    # Calculate age-specific measures in excess-mortality-period
                    curTotDataAge = dfCrisis[curAgeName+'_Data7DayMean'].sum()
                    curTotBaseAge = dfCrisis[curAgeName+'_Baseline'].sum()

                    curExcAge = curTotDataAge - curTotBaseAge
                    curExcPctAge = (curExcAge)/curTotBaseAge
                    # curExcPctAge = int(np.round(100 * curExcPctAge))
                    curExcPctAge = np.round(100 * curExcPctAge)


                    curRowToAdd['Exc_'+curAgeName] = curExcAge
                    curRowToAdd['Pct_'+curAgeName] = curExcPctAge
                    curRowToAdd['DataSum_'+curAgeName] = curTotDataAge
                    curRowToAdd['Baseline_'+curAgeName] = curTotBaseAge
                    
                # Add row to primary dataframe
                dfCrisesCollectAge.loc[len(dfCrisesCollectAge)] = curRowToAdd
                                

100%|██████████| 27/27 [01:23<00:00,  3.10s/it, Amt=Nordborg Amt, Period=1, Total periods=2, Start=1867-09-22, End=1915-01-01]     


In [30]:
# # Remove crises in earliest period of data, since data is not realiable in this period
# dfCrisesCollect = dfCrisesCollect.drop(dfCrisesCollect[dfCrisesCollect.End < np.datetime64('1820-01-01')].index)
dfCrisesCollectAge = dfCrisesCollectAge.drop(dfCrisesCollectAge[dfCrisesCollectAge.End < np.datetime64('1820-01-01')].index)

In [31]:
# # Sort by total excess
# dfCrisesCollect = dfCrisesCollect.sort_values('Excess',ascending=False)
dfCrisesCollectAge = dfCrisesCollectAge.sort_values('Excess',ascending=False)

In [32]:
# # Save results
# dfCrisesCollect.to_csv(pathResultsUpper + 'AllCrises_noAge_preClustering.csv')
# dfCrisesCollectAge.to_csv(pathResultsUpper + 'AllCrises_preClustering.csv')
dfCrisesCollectAge.to_csv(pathData + finalResultsFilename+'.csv',index=False)


In [33]:
# Also save file as excel (with dates as strings to avoid excel-problems)
dfCrisesCollectAgeExcel = dfCrisesCollectAge.copy()
dfCrisesCollectAgeExcel['Start'] = dfCrisesCollectAgeExcel['Start'].astype(str)
dfCrisesCollectAgeExcel['End'] = dfCrisesCollectAgeExcel['End'].astype(str)
dfCrisesCollectAgeExcel['DayWithMostBurials'] = dfCrisesCollectAgeExcel['DayWithMostBurials'].astype(str)

dfCrisesCollectAgeExcel.to_excel(pathData + finalResultsFilename+'.xlsx',index=False)

# Old version based on separate directories for each age-group

In [37]:
# dfCrisesCollectAge.head()

In [34]:
# # Prepare progressbar and go through all counties
# pbar = tqdm(allAmtIDs)
# for curAmtID in pbar:

#     # Get county name, data and periods
#     curAmtName = ps.getAmtName(curAmtID)
#     alldfs,allStarts,allEnds = ps.getAmtCollections(curAmtID)


#     # Go through each possible period
#     for i in range(len(allStarts)):

#         # Get the dataframe, start date and end date
#         curdf = alldfs[i].copy()
#         curStart = allStarts[i]
#         curEnd = allEnds[i]

#         # Make sure that the "Date" columns in the data-dataframe is a datetime64 object
#         curdf['Date'] = pd.to_datetime(curdf['Date'])

#         # Update progressbar
#         pbar.set_postfix(
#             {
#                 'Amt':curAmtName,
#                 'Period':i,
#                 'Total periods':len(allStarts),
#                 'Start':curStart,
#                 'End':curEnd,
#             }
#         )

#         # Determine filename results-file
#         curFileName =  str(int(curAmtID)) + '_'+curAmtName + '_'+pd.to_datetime(curStart).strftime('%Y-%m-%d') +'_'+pd.to_datetime(curEnd).strftime('%Y-%m-%d')
#         curFileName = curFileName + '_Total.csv' # Use results from analysis of all ages

#         # Load analysis file
#         dfPeriod = pd.read_csv(pathResultsUpper+'Age_Total/'+curFileName)
#         # Make sure date is a datetime64 object
#         dfPeriod['Date'] = pd.to_datetime(dfPeriod.Date)

#         # Restrict to valid period 
#         dfPeriod = dfPeriod[(dfPeriod.Date >= curStart) & (dfPeriod.Date < curEnd)].reset_index(drop=True)

#         # Get results and re-calculate excess
#         curTime = dfPeriod.Date 
#         curExcess = dfPeriod.DataSmooth - dfPeriod.Baseline 
#         curZscore = dfPeriod.Zscore 
        
#         # Use function from ExcessMortalityFunctions to determine mortality crisis period
#         dateGroups,allExcess  = emf.determineMortalityCrisis(curTime,curExcess,curZscore,upperThreshold=thresholdExcess,lowerThreshold=thresholdLower,maxDaysBelowThreshold=maxDaysBelowThreshold,minimumLengthOfEpidemic=minimumLengthOfEpidemic,returnExcessCount=True)
        
#         # Go through each mortality crisis
#         for excID in range(len(dateGroups)):
            
#             # Get current start, end and total excess
#             curGroup = dateGroups[excID]
#             curExc = allExcess[excID]

#             # If the period is significant enough
#             if (curExc >= excessCountThreshold):
                
#                 # Get start and end
#                 curCrisisStart = curGroup[0]
#                 curCrisisEnd = curGroup[1]

#                 # Calculate gender ratio of all deaths in period from data-collection dataframe
#                 dfDataCrisis = curdf[(curdf.Date >= curCrisisStart) & (curdf.Date <= curCrisisEnd)]
#                 GenderRatioInPeriod = dfDataCrisis.Male.sum()/(dfDataCrisis.Male.sum()+dfDataCrisis.Female.sum())
                
#                 # Determine duraction (in days)
#                 curDuration = int((curCrisisEnd - curCrisisStart)/np.timedelta64(1,'D'))

#                 # Make a dataframe consisting of analysis-results only during period
#                 dfCrisis = dfPeriod[(dfPeriod.Date >= curCrisisStart) & (dfPeriod.Date <= curCrisisEnd)]

#                 # Determine the date (during the crisis) where raw data is highest.
#                 curDeadliestDay = dfCrisis.iloc[dfCrisis.Data.argmax()]['Date']

#                 # Total deaths in period, data
#                 curTotData = dfCrisis.DataSmooth.sum()
#                 # Total deaths in period, baseline
#                 curTotBase = dfCrisis.Baseline.sum()
#                 # Excess deaths in entire period, in percent
#                 curExcPct = (curTotData - curTotBase)/curTotBase
#                 curExcPct = int(np.round(100 * curExcPct))

#                 # Get population estimate (rounded to an integer)
#                 _,populationInterpolated = AmtIDToDailyPopulation(curAmtID,curDeadliestDay)
#                 curPopulation = np.round(populationInterpolated.loc[curDeadliestDay]['Population'])

#                 # Collect results as a row to add to dataframe
#                 curRowToAdd = pd.Series({
#                     'Amt': curAmtName,
#                     'Start': curCrisisStart,
#                     'End': curCrisisEnd,
#                     'NumberOfDays': curDuration,
#                     'DayWithMostBurials': curDeadliestDay,
#                     'Excess': int(np.round(curExc)),
#                     'ExcessPct': curExcPct,
#                     'GenderRatio': GenderRatioInPeriod,
#                     'TimeOfYear': getQuarter(curDeadliestDay),
#                     'Season': getSeason(curDeadliestDay),
#                     'PopulationEstimate': curPopulation,
#                 })

#                 # # Add row to primary dataframe
#                 # dfCrisesCollect.loc[len(dfCrisesCollect)] = curRowToAdd
            

#                 #### Get results of age-specific analysis
#                 # Go through each agegroup
#                 for ageIndex in range(len(ageGroups)):
                    
#                     # Get the agegroup and the name of the group
#                     curAgeGroup = ageGroups[ageIndex]
#                     curAgeName = ageGroupNames[ageIndex]

#                     # Get the filename to load
#                     curFileName =  str(int(curAmtID)) + '_'+curAmtName + '_'+pd.to_datetime(curStart).strftime('%Y-%m-%d') +'_'+pd.to_datetime(curEnd).strftime('%Y-%m-%d')
#                     curFileName = curFileName + '_'+curAgeName+'.csv'

#                     # Load file
#                     dfPeriodAge = pd.read_csv(pathResultsUpper+'/Age_'+curAgeName+'/'+curFileName)
#                     # Convert to datetime
#                     dfPeriodAge['Date'] = pd.to_datetime(dfPeriodAge.Date)
                        
#                     # Calculate age-specific measures in excess-mortality-period
#                     dfCrisisAge = dfPeriodAge[(dfPeriodAge.Date >= curCrisisStart) & (dfPeriodAge.Date <= curCrisisEnd)]
#                     curTotDataAge = dfCrisisAge.DataSmooth.sum()
#                     curTotBaseAge = dfCrisisAge.Baseline.sum()
#                     curExcAge = curTotDataAge - curTotBaseAge
#                     curExcPctAge = (curExcAge)/curTotBaseAge
#                     curExcPctAge = int(np.round(100 * curExcPctAge))


#                     curRowToAdd['Exc_'+curAgeName] = curExcAge
#                     curRowToAdd['Pct_'+curAgeName] = curExcPctAge
#                     curRowToAdd['Total_'+curAgeName] = curTotDataAge
#                     curRowToAdd['Baseline_'+curAgeName] = curTotBaseAge
                    
#                 # Add row to primary dataframe
#                 dfCrisesCollectAge.loc[len(dfCrisesCollectAge)] = curRowToAdd
                                